In [32]:
# Dependencies and setup
import pandas as pd
import os

# csv paths
kaggle_path = "Resources/kaggle_clean.csv"
dow_path = "Resources/dow_clean.csv"
indu_path = "Resources/indu_clean.csv"
spy_path = "Resources/spy_clean.csv"

# Read the csv files into dataframes
kaggle_path = pd.read_csv(kaggle_path)
dow_path = pd.read_csv(dow_path)
indu_path = pd.read_csv(indu_path)
spy_path = pd.read_csv(spy_path)

In [33]:
# Create list of csvs
all_csvs = [kaggle_path, dow_path, indu_path, spy_path]

In [34]:
for df in csvs:   
    columns = df.columns.tolist()
    columns = ['symbol'] + [col for col in columns if col != 'symbol']
    df = df.sort_values(by="date", ascending=True)
    df = df[columns]
    df = df.round(2)
    print(f"{df.head()}")

      symbol        date   open   high    low  close    volume
0       NVDA  2014-06-02   4.76   4.80   4.70   4.74  18150000
10056   PANW  2014-06-02  25.04  25.06  24.00  24.56   6110100
12571    NOW  2014-06-02  52.33  52.34  49.46  50.26   3069100
2515    META  2014-06-02  63.23  63.59  62.05  63.08  35996000
15086    AMD  2014-06-02   4.03   4.03   3.94   3.97  10859500
     symbol        date   open   high    low  close   volume
1309    DOW  2019-03-20  52.75  53.50  49.50  49.80  2350838
1308    DOW  2019-03-21  49.99  50.00  48.20  48.98  1764671
1307    DOW  2019-03-22  48.80  49.95  48.16  48.60   844690
1306    DOW  2019-03-25  48.60  49.40  48.00  49.15   440892
1305    DOW  2019-03-26  49.00  49.75  48.18  48.85   504734
     symbol        date  open  high  low  close  volume
1704   INDU  2017-08-21  9.81  9.81  9.7   9.70   52682
1703   INDU  2017-08-22  9.70  9.70  9.7   9.70   27295
1702   INDU  2017-08-23  9.70  9.70  9.7   9.70       5
1701   INDU  2017-08-24  9.70  9

In [35]:
# Combine kaggle data and index csvs into one csv
master_df = pd.concat(all_csvs, ignore_index=True)
master_df


,symbol,date,open,high,low,close,volume
0,NVDA,2014-06-02,4.7650,4.795,4.6950,4.7350,18150000
1,NVDA,2014-06-03,4.7450,4.750,4.6525,4.7150,24321200
2,NVDA,2014-06-04,4.7025,4.755,4.7000,4.7200,16123600
3,NVDA,2014-06-05,4.7275,4.755,4.7000,4.7400,18869600
4,NVDA,2014-06-06,4.7625,4.800,4.7450,4.7575,16105200
...,...,...,...,...,...,...,...
26483,SPY,2017-08-28,245.1700,245.200,244.0900,244.5700,40565606
26484,SPY,2017-08-25,244.9000,245.610,244.3900,244.5600,64445912
26485,SPY,2017-08-24,245.0000,245.180,243.7500,243.9900,50741671
26486,SPY,2017-08-23,244.3300,245.050,244.1600,244.5600,50203837


In [36]:
#define path to Resources folder
resources_path = "Resources"

file_path = os.path.join(resources_path, "master.csv")
master_df.to_csv(file_path, index=False, header=True)